In [2]:
import pandas as pd
import random
import re

# tsv 파일을 DataFrame으로 읽기
user = pd.read_csv('user.tsv', sep='\t', names=['User', 'History', 'Train', 'Test'])
user_raw = pd.read_csv('user(raw).tsv', sep='\t',  names=['User', 'History', 'Train', 'Test'])

history_news = pd.read_csv('history/news.tsv', sep='\t',  names=['News ID', 'Publish', 'Title', 'Click time history', 'Category'])

train_news = pd.read_csv('train/news.tsv', sep='\t', names=['News ID', 'Publish', 'Title', 'Click time history', 'Category'])
train_behaviors = pd.read_csv('train/behaviors.tsv', sep='\t', names=['User ID', 'Click time', 'Click history', 'click'])

test_news = pd.read_csv('test/news.tsv', sep='\t', names=['News ID', 'Publish', 'Title', 'Click time history', 'Category'])
test_behaviors = pd.read_csv('test/behaviors.tsv', sep='\t', names=['User ID', 'Click time', 'Click history', 'click'])



In [3]:
# publish 순서에 맞게 오름차순으로 정렬
history_news_sorted = history_news.sort_values(by='Publish', ascending=True).reset_index(drop=True)
# publish 순서에 맞게 오름차순으로 정렬
train_news_sorted = train_news.sort_values(by='Publish', ascending=True).reset_index(drop=True)

In [108]:
ID = 'U30'

history = user[user['User']==ID]['History'].iloc[0]

# History에서 News ID 추출
news_ids = []
[news_ids.append(entry.split(',')[0]) for entry in history.split(';') if entry and entry.split(',')[0] not in news_ids]

# News ID의 맞는 Title 추출
titles = []
for news_id in news_ids:
    matching_rows = history_news[history_news['News ID'] == news_id]
    if not matching_rows.empty:
        titles.append(matching_rows.iloc[0]['Title'])
        


train = user[user['User']==ID]['Train'].iloc[0]

# Train에서 News ID 추출
train_ids = []
[train_ids.append(entry.split(',')[0]) for entry in train.split(';') if entry and entry.split(',')[0] not in train_ids]

# News ID의 맞는 Title 추출
train_titles = []
for train_id in train_ids:
    matching_rows = train_news[train_news['News ID'] == train_id]
    if not matching_rows.empty:
        train_titles.append(matching_rows.iloc[0]['Title'])


# news_id에서 각 news_id에 대한 유사한 publish 시간 뉴스 기사 찾기
negative_ids = []
negative_titles = []
used_ids = set(news_ids)  # 중복을 방지하기 위해 사용된 뉴스 ID를 추적

for news_id in news_ids:
    idx = history_news_sorted[history_news_sorted['News ID'] == news_id].index[0]
    above_idx, below_idx = idx - 1, idx + 1
    similar_ids = []
    similar_titles = []

    # 이미 존재하는 news_id와 중복을 제외하고 위에서 가장 가까운 뉴스를 찾기
    while above_idx >= 0:
        if history_news_sorted.loc[above_idx, 'News ID'] not in used_ids:
            similar_ids.append(history_news_sorted.loc[above_idx, 'News ID'])
            similar_titles.append(history_news_sorted.loc[above_idx, 'Title'])
            used_ids.add(history_news_sorted.loc[above_idx, 'News ID'])
            break
        above_idx -= 1

    # 이미 존재하는 news_id와 중복을 제외하고 아래에서 가장 가까운 뉴스를 찾기
    while below_idx < len(history_news_sorted):
        if history_news_sorted.loc[below_idx, 'News ID'] not in used_ids:
            similar_ids.append(history_news_sorted.loc[below_idx, 'News ID'])
            similar_titles.append(history_news_sorted.loc[below_idx, 'Title'])
            used_ids.add(history_news_sorted.loc[below_idx, 'News ID'])
            break
        below_idx += 1

    negative_ids.append(similar_ids)
    negative_titles.append(similar_titles)


# news_id에서 각 news_id에 대한 유사한 publish 시간 뉴스 기사 찾기
negative_train_ids = []
negative_train_titles = []
used_ids = set(train_ids)  # 중복을 방지하기 위해 used_ids에 train_id를 추가

for train_id in train_ids:
    idx = train_news_sorted[train_news_sorted['News ID'] == train_id].index[0] 
    # print(idx)
    above_idx, below_idx = idx - 1, idx + 1
    similar_ids = []
    similar_titles = []

    # Find the two closest news above, excluding already present news_ids and duplicates
    while above_idx >= 0 and len(similar_ids) < 2:
        if train_news_sorted.loc[above_idx, 'News ID'] not in used_ids:
            similar_ids.append(train_news_sorted.loc[above_idx, 'News ID'])
            similar_titles.append(train_news_sorted.loc[above_idx, 'Title'])
            used_ids.add(train_news_sorted.loc[above_idx, 'News ID'])
        above_idx -= 1

    # Find the two closest news below, excluding already present news_ids and duplicates
    while below_idx < len(train_news_sorted) and len(similar_ids) < 4:
        if train_news_sorted.loc[below_idx, 'News ID'] not in used_ids:
            similar_ids.append(train_news_sorted.loc[below_idx, 'News ID'])
            similar_titles.append(train_news_sorted.loc[below_idx, 'Title'])
            used_ids.add(train_news_sorted.loc[below_idx, 'News ID'])
        below_idx += 1

    negative_train_ids.append(similar_ids)
    negative_train_titles.append(similar_titles)


question_ids = [negative_list + [title] for negative_list, title in zip(negative_train_ids, train_ids)]
question_titles = [negative_list + [title] for negative_list, title in zip(negative_train_titles, train_titles)]

hidden_positions = []

# 각 행을 섞고 히든 값의 위치를 저장
for row in question_titles:
    hidden_value = row[-1]  # 히든 값 (마지막 요소)
    random.shuffle(row)  # 전체 행을 섞기
    
    hidden_index = row.index(hidden_value)  # 히든 값의 새로운 위치 찾기
    hidden_positions.append(hidden_index+1)  # 히든 값의 위치 저장


number = len(titles)

print('[Click History]')
for i in range(number):
    print(f'{i+1}. click : {titles[i]} ')
    
print()
print("Rank the five candidates for each question based on the user's interest in previously read articles.")

for i, titles in enumerate(question_titles):
    print(f"Question {i+1}) ", end="")
    for j, title in enumerate(titles):
        # print(f"{j+1}: {title} / ", end="")
        if j == len(titles) - 1:
            print(f"{j+1}: {title}", end="")
        else:
            print(f"{j+1}: {title} / ", end="")
    print()

print("\nDo not explain reasons in the response, just return a list of numbers for each article.")


[Click History]
1. click : Se lesernes nyttårsbilder 
2. click : Brannutrykning til kjøkken på Singsaker 
3. click : Emil Iversen sender bilder til Ustjugov på nettet 
4. click : Hvem syns du er Årets trønder? 
5. click : - Fast ansatte er et konkurransefortrinn 
6. click : Slik gikk det da elbilene måtte betale p-avgift 
7. click : - Jeg fikk svingt unna i siste liten 
8. click : Trafikkulykke på E14 
9. click : Reitan etablerer ny bensinstasjonkjede 

Rank the five candidates for each question based on the user's interest in previously read articles.
Question 1) 1: Frykter at Æ sender Rema inn i dødsspiral / 2: Forbrukerrådet advarer mot dorullselger / 3: Dette norske stedet er med på prestisjefylt internasjonal reiseliste / 4: Han kan være Rosenborgs nye ving / 5: Datatilsynet gransker butikkers kundeordninger

Do not explain reasons in the response, just return a list of numbers for each article.


In [109]:
questions = input("질문을 입력하세요: ")

# 질문에서 숫자를 추출
def extract_numbers(questions):
    # pattern = r"Question \d+ : ([\d, ]+)"
    pattern = r"Question \d+: ([\d, ]+)"
    return [list(map(int, num.split(', '))) for num in re.findall(pattern, questions)]

numbers_list = extract_numbers(questions)

correct_count = 0
total_diff = 0

diff_count = 0
# 각 질문의 숫자 목록에서 hidden_positions 값의 위치 찾기
for i, (hidden, numbers) in enumerate(zip(hidden_positions, numbers_list)):
    if numbers[0] == hidden:
        print(f"Question {i + 1} : Correct")
        correct_count += 1
    else:
        position = numbers.index(hidden)
        total_diff += abs(position)
        print(f"Question {i + 1} : {position + 1}")
    diff_count += 1

# 결과 요약 출력
print(f"\n총 Correct : {correct_count}/{len(hidden_positions)}개 ({correct_count / len(hidden_positions):.3%})")
if diff_count > 0:
    print(f"평균 오차범위 : {total_diff / diff_count:.3f}")


Question 1 : Correct

총 Correct : 1/1개 (100.000%)
평균 오차범위 : 0.000


In [103]:
print("\n##########################################\n")
# 정답
for i, label in enumerate(hidden_positions):
    print(f"Question Answer {i+1} : {label}")


##########################################

Question Answer 1 : 5
